In [1]:
import numpy as np
import pandas as pd

from util_io import init, finalize, dump_conf
from util_cost import cal_total, n_people, family_id_days_to_pref_cost
from util_check import deep_check

In [2]:
# constants #
N_families = 5000
N_days = 100
# constants #

# params #
path_init_conf =     '../output/m08-improved.csv'
path_dump_improved = '../output/m09-improved.csv' # lowest cost

In [3]:
assigned_day, family_on_day, occupancy = init(path_conf=path_init_conf)
etotal_low = cal_total(assigned_day, occupancy)
print('Init config cost:', etotal_low)

Read initial configs...
Read config completed.
Init config cost: 73646.92733343587


In [4]:
len(assigned_day)

5000

In [5]:
from docplex.mp.model import Model

In [6]:
model = Model('')

In [7]:
model.context.cplex_parameters.threads = 8

In [8]:
families = range(N_families)
days = range(1, N_days + 1)

In [9]:
# Variables
assignment_matrix = model.binary_var_matrix(families, days, 'x')

In [10]:
type(assignment_matrix), len(assignment_matrix)

(dict, 500000)

In [11]:
assignment_matrix[0, 1]  # check what is inside

docplex.mp.Var(type=B,name='x_0_1')

In [12]:
# Constraints
# each family only take one day
model.add_constraints_([
    model.sum([assignment_matrix[family, day] 
               for day in days]) == 1 for family in families
])

In [13]:
model.add_constraints_([
    model.sum(
        [assignment_matrix[family, day] * n_people[family] for family in families]
    ) <= 300
    for day in days
])
model.add_constraints_([
    model.sum(
        [assignment_matrix[family, day] * n_people[family] for family in families]
    ) >= 125
    for day in days
])

In [14]:
# Objective - Preference cost only as approximation
model.minimize(
    model.sum([
        assignment_matrix[family, day] * family_id_days_to_pref_cost[family, day]
        for family in families for day in days
    ])
)

In [15]:
%%time
# Solve
sol = model.solve()

DOcplexLimitsExceeded: **** Promotional version. Problem size limits exceeded, CPLEX code=1016

In [16]:
print('Total cost = ', model.Objective().Value())

AttributeError: 'Model' object has no attribute 'Objective'

In [ ]:
print("Time = ", model.WallTime(), " milliseconds")

In [ ]:
1